## Library

In [ ]:
import numpy as np
import pandas as pd
import zipfile
import os
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings(action="ignore")

DATA_PATH = "/kaggle/input/coupon-purchase-prediction/"

## EDA
* coupon_detail_train/test.csv 쿠폰 구매 정보</br>
  ITEM_COUNT: 쿠폰 개수</br>
  I_DATE: 구매정보</br>
  SMALL_AREA_NAME: 지역 정보</br>
  PURCHASEID_hash: 구매 해시값</br>
  USER_ID_hash: 유저 해시값</br>
  COUPON_ID_hash: 쿠폰 해시값</br>
  </br>
* coupon_list_train/test.csv 모든 쿠폰 정보</br>
  CAPSULE_TEXT: 장르</br>
  GENRE_NAME: 장르</br>
  PRICE_RATE: 원가 대비 할인율</br>
  CATALOG_PRICE: 원가</br>
  DISCOUNT_PRICE: 할인되는 금액</br>
  DISPFROM: 게시 시작날짜</br>
  DISPEND: 게시 종료날짜</br>
  DISPPERIOD: 게시기간</br>
  VALIDFROM: 사용가능 시작날짜</br>
  VALIDEND: 사용가능 종료날짜</br>
  VALIDPERIOD: 사용가능 기간</br>
  USABLE_DATE_MON: 월요일 사용가능 여부</br>
  USABLE_DATE_TUE: 화요일 사용가능 여부</br>
  USABLE_DATE_WED: 수요일 사용가능 여부</br>
  USABLE_DATE_THU: 목요일 사용가능 여부</br>
  USABLE_DATE_FRI: 금요일 사용가능 여부</br>
  USABLE_DATE_SAT: 토요일 사용가능 여부</br>
  USABLE_DATE_SUN: 일요일 사용가능 여부</br>
  USABLE_DATE_HOLIDAY: 공유힐 사용가능 여부</br>
  USABLE_DATE_BEFORE_HOLIDAY: 공휴일 외 사용가능 여부</br>
  large_area_name: 사용가능 지역</br>
  ken_name</br>
  small_area_name</br>
  COUPON_ID_hash: 쿠폰 해시값(PK)</br>
  </br>
* user_list.csv 회원 정보</br>
  REG_DATE: 등록일자</br>
  SEX_ID: 성별</br>
  AGE: 나이</br>
  WITHDRAW_DATE: 탈퇴일자</br>
  PREF_NAME: 선호지역</br>
  USER_ID_hash: 유저 해시값(PK)</br>
  </br>
* coupon_visit_train/test.csv 사용자의 웹사이트 방문, 구매 기록</br>
  PURCHASE_FLG: 구매여부</br>
  I_DATE: 방문일시</br>
  PAGE_SERIAL: 방문 페이지 번호</br>
  REFERRER_hash: 방문 참조값 해시값 (방문전 링크값인거 같은데 활용 어떻게 할지 잘 모르겠음)</br>
  VIEW_COUPON_ID_hash: 확인한 쿠폰값 해시값</br>
  USER_ID_hash: 유저 해시값</br>
  SESSION_ID_hash: 세션 해시값</br>
  PURCHASEID_hash: 구매 해시값</br>

### data overview

In [ ]:
with zipfile.ZipFile(DATA_PATH + "coupon_detail_train.csv.zip", 'r') as zip_ref:
    # 압축 안의 파일 목록 확인
    file_list = zip_ref.namelist()
    print("압축 안의 파일:", file_list)
    # 첫 번째 CSV 파일을 DataFrame으로 읽기
    with zip_ref.open(file_list[0]) as file:
        df_detail = pd.read_csv(file)

# 결과 확인
df_detail.info()

In [ ]:
df_detail.head(1).T

In [ ]:
with zipfile.ZipFile(DATA_PATH + "coupon_list_train.csv.zip", 'r') as zip_ref:
    # 압축 안의 파일 목록 확인
    file_list = zip_ref.namelist()
    print("압축 안의 파일:", file_list)
    # 첫 번째 CSV 파일을 DataFrame으로 읽기
    with zip_ref.open(file_list[0]) as file:
        df_list = pd.read_csv(file)

# 결과 확인
df_list.info()


In [ ]:
df_list.head(1).T

In [ ]:
with zipfile.ZipFile(DATA_PATH + "coupon_visit_train.csv.zip", 'r') as zip_ref:
    # 압축 안의 파일 목록 확인
    file_list = zip_ref.namelist()
    print("압축 안의 파일:", file_list)
    # 첫 번째 CSV 파일을 DataFrame으로 읽기
    with zip_ref.open(file_list[0]) as file:
        df_visit = pd.read_csv(file)

# 결과 확인
df_visit.info()

In [ ]:
df_visit.head(1).T

In [ ]:
df_user.head(1).T

In [ ]:
with zipfile.ZipFile(DATA_PATH + "user_list.csv.zip", 'r') as zip_ref:
    # 압축 안의 파일 목록 확인
    file_list = zip_ref.namelist()
    print("압축 안의 파일:", file_list)
    # 첫 번째 CSV 파일을 DataFrame으로 읽기
    with zip_ref.open(file_list[0]) as file:
        df_user = pd.read_csv(file)

# 결과 확인
df_user.info()

### 결측치 확인

In [ ]:
print(f"<구매정보>\n{df_detail.isnull().sum()}\n")
print(f"<전체쿠폰정보>\n{df_list.isnull().sum()}\n")
print(f"<유저방문정보>\n{df_visit.isnull().sum()}\n")
print(f"<고객정보>\n{df_user.isnull().sum()}\n")

In [ ]:
print(df_detail.shape)
print(df_list.shape)
print(df_visit.shape)
print(df_user.shape)

## Preprocessing

In [ ]:
df_detail['I_DATE'] = pd.to_datetime(df_detail['I_DATE'])
df_detail['I_MONTH'] = df_detail['I_DATE'].dt.month
df_detail['I_DATE'] = df_detail['I_DATE'].dt.date

In [ ]:
df_detail.info()

In [ ]:
df_list['PURCHASE_PRICE'] = df_list['CATALOG_PRICE'] - df_list['DISCOUNT_PRICE']

In [ ]:
df_list.head(3)

### Merging: df_detail(구매기록) + df_list(쿠폰정보)

In [ ]:
df_detail.head(3)

In [ ]:
df_list.head(3)

In [ ]:
# PK확인
df_list.shape[0], len(df_list['COUPON_ID_hash'].unique())

In [ ]:
# PK확인
df_detail.shape[0], len(df_detail['PURCHASEID_hash'].unique())

In [ ]:
df = pd.merge(
    df_detail[['ITEM_COUNT','I_DATE','I_MONTH','SMALL_AREA_NAME','USER_ID_hash','COUPON_ID_hash', 'PURCHASEID_hash']],
    df_list[['GENRE_NAME','PRICE_RATE','PURCHASE_PRICE','COUPON_ID_hash']],
    on='COUPON_ID_hash',
    how='left'
)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df_detail['ITEM_COUNT'].max()

In [ ]:
# PK확인
df_user.shape[0], len(df_user['USER_ID_hash'].unique())

In [ ]:
df = pd.merge(
    df,
    df_user[['SEX_ID','AGE','USER_ID_hash']],
    on='USER_ID_hash',
    how='left'
)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

## RFM

In [ ]:
last = df['I_DATE'].max() + pd.DateOffset(days=1) # 마지막 거래일자 다음날을 기준일로 설정

rfm_df = df.groupby(['USER_ID_hash']).agg({
    'I_DATE': lambda x:(last-pd.to_datetime(x.max())).days,
    'PURCHASEID_hash': lambda x:x.nunique(),
    'PURCHASE_PRICE': sum
})
rfm_df.rename(columns={'거래날짜':'Recency', '거래':'Frequency', '지불금액':'Monetary'},inplace=True)

rfm_df.head(3)

In [ ]:
last = df['I_DATE'].max() + pd.DateOffset(days=1) # 마지막 거래일자 다음날을 기준일로 설정

rfm_df = df.groupby(['USER_ID_hash']).agg({
    'I_DATE': lambda x:(last-pd.to_datetime(x.max())).days,
    'PURCHASEID_hash': lambda x:x.nunique(),
    'PURCHASE_PRICE': sum
})
rfm_df.rename(columns={'I_DATE':'Recency', 'PURCHASEID_hash':'Frequency', 'PURCHASE_PRICE':'Monetary'},inplace=True)

rfm_df.head(3)

In [ ]:
rfm_df.shape

### RFM visualization

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(data=rfm_df, x='Recency', bins=30, kde=False)
plt.title('Recency Distribution')
plt.xlabel('Recency (days)')
plt.ylabel('Count')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(data=rfm_df, x='Frequency', bins=20, kde=False)
plt.title('Frequency Distribution')
plt.xlabel('Frequency')
plt.ylabel('Count')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(4,8))
sns.boxplot(data=rfm_df, y='Frequency')
plt.ylabel('count')
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(data=rfm_df, x='Monetary', bins=20, kde=False)
plt.title('Monetary Distribution')
plt.xlabel('spend amount')
plt.ylabel('Count')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(4,8))
sns.boxplot(data=rfm_df, y='Monetary')
plt.ylabel('count')
plt.show()

### recency grading

In [ ]:
def assign_R(recency):
    if recency <= 30:
        return 5
    elif recency <= 60:
        return 4
    elif recency <= 90:
        return 3
    elif recency <= 180:
        return 2
    elif recency <= 365:
        return 1
    else:
        return 0

# rfm_df['R'] = assign_R(rfm['Recency'])는 틀림. 함수에 Recency값 전체를 넣기 때문
rfm_df['R'] = rfm_df['Recency'].apply(assign_R)

In [ ]:
rfm_df.head()